# Graphs

This notebook implements a generic graph that can be:
- Directed or undirected
- Weighted or unweighted

Representation:
- Adjacency list stored as a HashMap of HashMaps
- For unweighted graphs, edges use weight `1.0` by default

Included operations:
- Structure: add/remove vertices and edges, query neighbors/edges
- Traversals: BFS, DFS
- Shortest paths: Dijkstra (for non-negative weights)

We'll add a `SparseGraph<T>` struct with a clean API and sample usage for both undirected/unweighted and directed/weighted graphs.

In [ ]:
use std::collections::{HashMap, HashSet, VecDeque, BinaryHeap};
use std::hash::Hash;
use std::cmp::Ordering;

// SparseGraph represents an adjacency-list based graph
#[derive(Debug, Clone)]
pub struct SparseGraph<T> {
    directed: bool,
    weighted: bool,
    adj: HashMap<T, HashMap<T, f64>>, // adjacency list: vertex -> neighbors -> weight
}

impl<T> SparseGraph<T>
where
    T: Clone + Eq + Hash + PartialOrd,
{
    // Create a new sparse graph
    pub fn new(directed: bool, weighted: bool) -> Self {
        SparseGraph {
            directed,
            weighted,
            adj: HashMap::new(),
        }
    }

    // Add a vertex if it doesn't exist
    pub fn add_vertex(&mut self, v: T) {
        self.adj.entry(v).or_insert_with(HashMap::new);
    }

    // Add an edge between two vertices
    pub fn add_edge(&mut self, u: T, v: T, weight: f64) {
        self.add_vertex(u.clone());
        self.add_vertex(v.clone());
        
        let w = if self.weighted { weight } else { 1.0 };
        
        self.adj.get_mut(&u).unwrap().insert(v.clone(), w);
        if !self.directed {
            self.adj.get_mut(&v).unwrap().insert(u, w);
        }
    }

    // Check if an edge exists
    pub fn has_edge(&self, u: &T, v: &T) -> bool {
        if let Some(neighbors) = self.adj.get(u) {
            neighbors.contains_key(v)
        } else {
            false
        }
    }

    // Get the weight of an edge
    pub fn get_weight(&self, u: &T, v: &T) -> Option<f64> {
        self.adj.get(u)?.get(v).copied()
    }

    // Get all vertices
    pub fn vertices(&self) -> Vec<T> {
        self.adj.keys().cloned().collect()
    }

    // Get neighbors of a vertex
    pub fn neighbors(&self, v: &T) -> HashMap<T, f64> {
        self.adj.get(v).cloned().unwrap_or_default()
    }

    // Get all edges
    pub fn edges(&self) -> Vec<Edge<T>> {
        let mut edges = Vec::new();
        let mut seen = HashSet::new();
        
        for (u, neighbors) in &self.adj {
            for (v, &weight) in neighbors {
                if self.directed {
                    edges.push(Edge {
                        from: u.clone(),
                        to: v.clone(),
                        weight,
                    });
                } else {
                    // For undirected graphs, avoid duplicate edges
                    let key = if u <= v {
                        (u.clone(), v.clone())
                    } else {
                        (v.clone(), u.clone())
                    };
                    
                    if !seen.contains(&key) {
                        edges.push(Edge {
                            from: u.clone(),
                            to: v.clone(),
                            weight,
                        });
                        seen.insert(key);
                    }
                }
            }
        }
        edges
    }

    // Breadth-first search
    pub fn bfs(&self, start: &T) -> Vec<T> {
        if !self.adj.contains_key(start) {
            return Vec::new();
        }
        
        let mut visited = HashSet::new();
        let mut queue = VecDeque::new();
        let mut result = Vec::new();
        
        queue.push_back(start.clone());
        visited.insert(start.clone());
        
        while let Some(current) = queue.pop_front() {
            result.push(current.clone());
            
            if let Some(neighbors) = self.adj.get(&current) {
                for neighbor in neighbors.keys() {
                    if !visited.contains(neighbor) {
                        visited.insert(neighbor.clone());
                        queue.push_back(neighbor.clone());
                    }
                }
            }
        }
        result
    }

    // Depth-first search
    pub fn dfs(&self, start: &T) -> Vec<T> {
        if !self.adj.contains_key(start) {
            return Vec::new();
        }
        
        let mut visited = HashSet::new();
        let mut stack = vec![start.clone()];
        let mut result = Vec::new();
        
        while let Some(current) = stack.pop() {
            if visited.contains(&current) {
                continue;
            }
            
            visited.insert(current.clone());
            result.push(current.clone());
            
            if let Some(neighbors) = self.adj.get(&current) {
                let mut neighbors_vec: Vec<_> = neighbors.keys().collect();
                neighbors_vec.reverse(); // For consistent traversal order
                for neighbor in neighbors_vec {
                    if !visited.contains(neighbor) {
                        stack.push(neighbor.clone());
                    }
                }
            }
        }
        result
    }

    // Dijkstra's shortest path algorithm
    pub fn dijkstra(&self, start: &T) -> HashMap<T, f64> {
        if !self.adj.contains_key(start) {
            return HashMap::new();
        }
        
        let mut dist = HashMap::new();
        let mut heap = BinaryHeap::new();
        
        // Initialize distances
        for vertex in self.adj.keys() {
            dist.insert(vertex.clone(), f64::INFINITY);
        }
        dist.insert(start.clone(), 0.0);
        
        heap.push(State {
            vertex: start.clone(),
            cost: 0.0,
        });
        
        while let Some(State { vertex, cost }) = heap.pop() {
            // Skip if we've found a better path already
            if cost != *dist.get(&vertex).unwrap_or(&f64::INFINITY) {
                continue;
            }
            
            if let Some(neighbors) = self.adj.get(&vertex) {
                for (neighbor, &weight) in neighbors {
                    if weight < 0.0 {
                        panic!("Dijkstra does not support negative edge weights");
                    }
                    
                    let new_cost = cost + weight;
                    if new_cost < *dist.get(neighbor).unwrap_or(&f64::INFINITY) {
                        dist.insert(neighbor.clone(), new_cost);
                        heap.push(State {
                            vertex: neighbor.clone(),
                            cost: new_cost,
                        });
                    }
                }
            }
        }
        
        dist
    }

    // String representation
    pub fn description(&self) -> String {
        let kind = if self.directed { "Directed" } else { "Undirected" };
        let weight = if self.weighted { "Weighted" } else { "Unweighted" };
        format!(
            "SparseGraph {} {} |V|={} |E|={}",
            kind,
            weight,
            self.adj.len(),
            self.edges().len()
        )
    }
}

// Edge representation
#[derive(Debug, Clone)]
pub struct Edge<T> {
    pub from: T,
    pub to: T,
    pub weight: f64,
}

// State for Dijkstra's algorithm (for priority queue)
#[derive(Debug, Clone)]
struct State<T> {
    vertex: T,
    cost: f64,
}

impl<T> PartialEq for State<T>
where
    T: PartialEq,
{
    fn eq(&self, other: &Self) -> bool {
        self.cost.eq(&other.cost)
    }
}

impl<T> Eq for State<T> where T: Eq {}

impl<T> PartialOrd for State<T>
where
    T: PartialEq,
{
    fn partial_cmp(&self, other: &Self) -> Option<Ordering> {
        // Flip the ordering for min-heap behavior
        other.cost.partial_cmp(&self.cost)
    }
}

impl<T> Ord for State<T>
where
    T: Eq,
{
    fn cmp(&self, other: &Self) -> Ordering {
        self.partial_cmp(other).unwrap_or(Ordering::Equal)
    }
}

Error: binary operation `<=` cannot be applied to type `&T`

## Examples

Below are quick examples showing undirected/unweighted and directed/weighted graph usage.

In [ ]:
// Undirected, unweighted graph usage
let mut g = SparseGraph::new(false, false);
g.add_edge("A".to_string(), "B".to_string(), 0.0);
g.add_edge("A".to_string(), "C".to_string(), 0.0);
g.add_edge("B".to_string(), "D".to_string(), 0.0);
g.add_edge("C".to_string(), "D".to_string(), 0.0);

println!("{}", g.description());
println!("Vertices: {:?}", g.vertices());
println!("BFS from A: {:?}", g.bfs(&"A".to_string()));
println!("DFS from A: {:?}", g.dfs(&"A".to_string()));

In [ ]:
// Directed, weighted graph + Dijkstra
let mut gw = SparseGraph::new(true, true);
gw.add_edge("A".to_string(), "B".to_string(), 4.0);
gw.add_edge("A".to_string(), "C".to_string(), 2.0);
gw.add_edge("C".to_string(), "B".to_string(), 1.0);
gw.add_edge("B".to_string(), "D".to_string(), 5.0);
gw.add_edge("C".to_string(), "D".to_string(), 8.0);

println!("{}", gw.description());
println!("Dijkstra from A: {:?}", gw.dijkstra(&"A".to_string()));